In [1]:
# Install these two if you haven't already
import mysql.connector
import sqlalchemy

db = mysql.connector.connect(
    host=os.environ.get('DB_IP_ADDRESS'),  # Set environment variables before you do this
    user="hussen",
    passwd=os.environ.get('DB_PASSWORD_REMOTE'),  # Set environment variables before you do this
    database="testdatabase"
)

mycursor = db.cursor()



engine = sqlalchemy.create_engine(f"mysql+mysqlconnector://hussen:{os.environ.get('DB_PASSWORD_REMOTE')}@{os.environ.get('DB_IP_ADDRESS')}:3306/testdatabase")



# mycursor.execute('CREATE DATABASE IF NOT EXISTS testdatabase')


In [2]:
import os
import pandas as pd

from googleapiclient.discovery import build

api_key = os.environ.get("YT_API_KEY")

#@lru_cache
def get_comments_and_replies(videoId):

    youtube = build('youtube', 'v3', developerKey=api_key)

    request = youtube.commentThreads().list(
            part = 'snippet',
            videoId = videoId,
            order = 'relevance'
        )

    response = request.execute()

    comments = []

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']

            comments.append(comment)

        if 'nextPageToken' in response:
            request = youtube.commentThreads().list(
                    part = 'snippet',
                    videoId = videoId,
                    order = 'relevance',
                    pageToken = response['nextPageToken']
                )

            response = request.execute()            

        else:
            response = False


    return comments


In [3]:
df = pd.DataFrame(columns=['videoId', 'comments'])  # Creating an empty pandas dataframe
df['comments'] = get_comments_and_replies(videoId='TT95ZWbpa-I')  # Populating the 'comments' column
df.loc[:,'videoId'] = 'TT95ZWbpa-I'  # Poulating the 'videoID' coulmn

In [4]:
df.head()

,videoId,comments
0,TT95ZWbpa-I,Get 68% off NordVPN! Only $3.71/mo plus 1 mont...
1,TT95ZWbpa-I,impressive how she so thoughtfully embraces be...
2,TT95ZWbpa-I,Am I the only one who wants a whole movie with...
3,TT95ZWbpa-I,When your roommate studies abroad for a semest...
4,TT95ZWbpa-I,People say vegans are annoying but I’d say peo...


In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from textblob import TextBlob

stop_words = set(stopwords.words('english'))

def remove_stopwords(line):
    word_tokens = word_tokenize(line)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

df['comments'] = df['comments'].str.lower()
df['comments'] = df['comments'].str.strip()
df['stop_comments'] = df['comments'].apply(lambda x : remove_stopwords(x))
df['polarity'] = df['comments'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['pol_cat'] = df.apply(lambda _: '', axis=1)

df.head()

,videoId,comments,stop_comments,polarity,pol_cat
0,TT95ZWbpa-I,get 68% off nordvpn! only $3.71/mo plus 1 mont...,get 68 % nordvpn ! $ 3.71/mo plus 1 month free...,0.341667,
1,TT95ZWbpa-I,impressive how she so thoughtfully embraces be...,impressive thoughtfully embraces annoying . st...,0.150000,
2,TT95ZWbpa-I,am i the only one who wants a whole movie with...,one wants whole movie julie nolke playing char...,0.133333,
3,TT95ZWbpa-I,when your roommate studies abroad for a semest...,roommate studies abroad semester comes back ac...,0.000000,
4,TT95ZWbpa-I,people say vegans are annoying but i’d say peo...,people say vegans annoying ’ say people “ glob...,-0.100000,


In [13]:
df['pol_cat'][df['polarity'] > 0] = 1
df['pol_cat'][df['polarity'] <= 0] = -1

C:\Users\Hussen\AppData\Local\Temp\ipykernel_25560\625341433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pol_cat'][df['polarity'] > 0] = 1
C:\Users\Hussen\AppData\Local\Temp\ipykernel_25560\625341433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pol_cat'][df['polarity'] <= 0] = -1


In [14]:
df.head()

,videoId,comments,stop_comments,polarity,pol_cat
0,TT95ZWbpa-I,get 68% off nordvpn! only $3.71/mo plus 1 mont...,get 68 % nordvpn ! $ 3.71/mo plus 1 month free...,0.341667,1
1,TT95ZWbpa-I,impressive how she so thoughtfully embraces be...,impressive thoughtfully embraces annoying . st...,0.150000,1
2,TT95ZWbpa-I,am i the only one who wants a whole movie with...,one wants whole movie julie nolke playing char...,0.133333,1
3,TT95ZWbpa-I,when your roommate studies abroad for a semest...,roommate studies abroad semester comes back ac...,0.000000,-1
4,TT95ZWbpa-I,people say vegans are annoying but i’d say peo...,people say vegans annoying ’ say people “ glob...,-0.100000,-1


In [5]:
# Storing it in our database
df.to_sql(
    name='comments',
    con=engine,
    if_exists='replace',
    index=False
)

1929

In [6]:
# Reading what we stored just to see if it works
df2 = pd.read_sql_table('comments', con=engine)
df2.head()

,videoId,comments
0,TT95ZWbpa-I,Get 68% off NordVPN! Only $3.71/mo plus 1 mont...
1,TT95ZWbpa-I,impressive how she so thoughtfully embraces be...
2,TT95ZWbpa-I,Am I the only one who wants a whole movie with...
3,TT95ZWbpa-I,When your roommate studies abroad for a semest...
4,TT95ZWbpa-I,People say vegans are annoying but I’d say peo...


In [9]:
mycursor.execute("SHOW TABLES")

print(mycursor.fetchone())


('comments',)
